_Lambda School Data Science_

# Reshape data

Objectives
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- pandas documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

## Why reshape data?

#### Some libraries prefer data in different formats

For example, the Seaborn data visualization library prefers data in "Tidy" format often (but not always).

> "[Seaborn will be most powerful when your datasets have a particular organization.](https://seaborn.pydata.org/introduction.html#organizing-datasets) This format ia alternately called “long-form” or “tidy” data and is described in detail by Hadley Wickham. The rules can be simply stated:

> - Each variable is a column
- Each observation is a row

> A helpful mindset for determining whether your data are tidy is to think backwards from the plot you want to draw. From this perspective, a “variable” is something that will be assigned a role in the plot."

#### Data science is often about putting square pegs in round holes

Here's an inspiring [video clip from _Apollo 13_](https://www.youtube.com/watch?v=ry55--J4_VQ): “Invent a way to put a square peg in a round hole.” It's a good metaphor for data wrangling!

## Upgrade Seaborn

Run the cell below which upgrades Seaborn and automatically restarts your Google Colab Runtime.

In [0]:
!pip install seaborn --upgrade ## terminal command
import os
os.kill(os.getpid(), 9)  ## do this when upgrading a package onto colab..

    100% |████████████████████████████████| 215kB 24.4MB/s 
  Found existing installation: seaborn 0.7.1
    Uninstalling seaborn-0.7.1:
      Successfully uninstalled seaborn-0.7.1


## Hadley Wickham's Examples

From his paper, [Tidy Data](http://vita.had.co.nz/papers/tidy-data.html)

In [0]:
# magic command for a notebook environment.. in this case, render visualizations inline for matplotlib
%matplotlib inline  
import pandas as pd
import numpy as np
import seaborn as sns

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

"Table 1 provides some data about an imaginary experiment in a format commonly seen in the wild. 

The table has two columns and three rows, and both rows and columns are labelled."

In [0]:
table1

"There are many ways to structure the same underlying data. 

Table 2 shows the same data as Table 1, but the rows and columns have been transposed. The data is the same, but the layout is different."

In [0]:
table2

"Table 3 reorganises Table 1 to make the values, variables and obserations more clear.

Table 3 is the tidy version of Table 1. Each row represents an observation, the result of one treatment on one person, and each column is a variable."

| name         | trt | result |
|--------------|-----|--------|
| John Smith   | a   | -      |
| Jane Doe     | a   | 16     |
| Mary Johnson | a   | 3      |
| John Smith   | b   | 2      |
| Jane Doe     | b   | 11     |
| Mary Johnson | b   | 1      |

## Table 1 --> Tidy

We can use the pandas `melt` function to reshape Table 1 into Tidy format.

In [0]:
table1

In [0]:
table1.columns

In [0]:
table1.index # names are part of the index.. not part of a column. Important to check this!

In [0]:
table1.index.tolist()

In [0]:
table1.reset_index().melt() ## reset index first. then used melt function.. but not quite what we want yet.

In [0]:
tidy = table1.reset_index().melt(id_vars='index') # this is what we want by adding parameter to the melt function
## not that 'index' just happens to be the name of the column. but in the parameter, you refer to the name of the column
tidy

In [0]:
## let's tidy this dataset up even more!
tidy = tidy.rename(columns={  # renaming the columns
    'index': 'name',
    'variable': 'trt',
    'value': 'result'
})

tidy

In [0]:
tidy['trt'] = tidy['trt'].replace('treatment', '') # get rid of 'treatment' in the data for 'trt' column
tidy

In [0]:
#from numpy import nan
tidy['result'] = tidy['result'].replace(nan, '-')
tidy

***extra challenge*** 
change a and b into 0 and 1

In [0]:
# tidy['trt'].replace('a', 0).replace('b', 1) .. replace!


## tidy['trt'].map({a:0, b:1}) .. map!

## (tidy['trt'] == b).astype(int) ... returns booleans.. which are treated as 1's and 0's!

## tidy.apply(lambda x: ord(x) - ord('a')) .. get unicode and lambda fancy-like. wowowo

## Table 2 --> Tidy

In [0]:
## self exercise
table2

In [0]:
table2_tran = table2.T
table2_tran

In [0]:
tidy2 = table2_tran.reset_index().melt(id_vars='index')
tidy2

In [0]:
tidy2 = tidy2.rename(columns={
    'index': 'name',
    'variable': 'trt',
    'value': 'result'
})

tidy2

In [0]:
tidy2['trt'] = tidy2['trt'].str.replace('treatment', '')
tidy2


## Tidy --> Table 1

The `pivot_table` function is the inverse of `melt`.

In [0]:
table1

In [0]:
tidy

In [0]:
tidy.pivot_table(index='name', columns='trt', values='result')  # think of pivot and melt as opposite treatments of data

## Tidy --> Table 2

In [0]:
## self exercise
tidy2.pivot_table(index='name', columns='trt', values='result')

## Load Instacart data

Let's return to the dataset of [3 Million Instacart Orders](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)

If necessary, uncomment and run the cells below to re-download and extract the data

In [14]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-03-28 05:26:27--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.166.29
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.166.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  45.4MB/s    in 4.7s    

2019-03-28 05:26:32 (41.4 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [15]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


Run these cells to load the data

In [16]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01/instacart_2017_05_01


In [0]:
products = pd.read_csv('products.csv')

order_products = pd.concat([pd.read_csv('order_products__prior.csv'), 
                            pd.read_csv('order_products__train.csv')])

orders = pd.read_csv('orders.csv')

***Seaborn example***
- each variable is a column
- each observation is a row

In [0]:
sns.catplot(x='trt', y='result', col='name', kind='bar', data=tidy, height=2) #cool

## Goal: Reproduce part of this example

Instead of a plot with 50 products, we'll just do two — the first products from each list
- Half And Half Ultra Pasteurized
- Half Baked Frozen Yogurt

In [18]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*wKfV6OV-_1Ipwrl7AjjSuw.png'
example = Image(url=url, width=600)

display(example)

So, given a `product_name` we need to calculate its `order_hour_of_day` pattern.

## Subset and Merge

In [8]:
products.columns.tolist()

['product_id', 'product_name', 'aisle_id', 'department_id']

In [9]:
order_products.columns.tolist()

['order_id', 'product_id', 'add_to_cart_order', 'reordered']

In [10]:
orders.columns.tolist()

['order_id',
 'user_id',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order']

In [0]:
merged = (products[['product_id', 'product_name']]
          .merge(order_products[['order_id', 'product_id']])
          .merge(orders[['order_id', 'order_hour_of_day']]))


In [12]:
products.shape, order_products.shape, orders.shape, merged.shape

((49688, 4), (33819106, 4), (3421083, 7), (33819106, 4))

In [13]:
merged.head()

,product_id,product_name,order_id,order_hour_of_day
0,1,Chocolate Sandwich Cookies,1107,11
1,769,Sliced American Cheese,1107,11
2,6184,Clementines,1107,11
3,8048,Packaged Grape Tomatoes,1107,11
4,9007,Frosted Flakes,1107,11


In [0]:
# what condition will filter 'merged' to just the two products that we care about?
#condition = ((merged['product_name']=='Half Baked Frozen Yogurt') |
            #merged['product_name']=='Half And Half Ultra Pasteurized')

product_times = merged.groupby('product_name')['order_hour_of_day'].mean()
product_times = product_times.reset_index()
product_times.columns= ['product_name', 'average_time']




In [0]:
product_sales = merged['product_name'].value_counts()
product_sales = product_sales.reset_index()
product_sales.columns = ['product_name', 'total sales']


In [0]:
times_and_sales = pd.merge(product_times, product_sales)

In [0]:
popular = times_and_sales[times_and_sales['total sales'] > 2900]
popular_evening = popular.sort_values(by='average_time', ascending=False).head(25)


In [0]:
popular = times_and_sales[times_and_sales['total sales'] > 2900]
popular_morning = popular.sort_values(by='average_time', ascending=True).head(25)

## 4 ways to reshape and plot

### 1. value_counts

In [36]:
froyo = subset[subset['product_name'] == 'Half Baked Frozen Yogurt']
cream = subset[subset['product_name']== 'Half And Half Ultra Pasteurized']

NameError: ignored

In [0]:
(cream['order_hour_of_day']
.value_counts(normalize=True)
.sort_index()
.plot())
(froyo['order_hour_of_day']
.value_counts(normalize=True)
.sort_index()
.plot())

### 2. crosstab

In [34]:
pd.crosstab(subset['order_hour_of_day'],
           subset['product_name'],
           normalize='columns').plot()


NameError: ignored

### 3. pivot_table

In [0]:
subset.pivot_table(index='order_hour_of_day',
                  columns='product_name',
                  values='order_id',
                  aggfunc=len).plot()

### 4. melt

In [0]:
table = pd.crosstab(subset['order_hour_of_day'],
                   subset['product_name'],
                   normalize=True)

melted = table.reset_index().melt(id_vars='order_hour_of_day').rename(columns={'order_hour_of_day': 'Hour of Day Ordered',
                                                                     'product_name': 'Product',
                                                                     'value': 'Percent of Orders by Product'})

sns.relplot(x='Hour of Day Ordered', y='Percent of Orders by Product', hue="Product", data=melted, kind='line')

# ASSIGNMENT
- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
  - Table 2 --> Tidy
  - Tidy --> Table 2

- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
flights = sns.load_dataset('flights')

In [0]:
flights.head()

In [0]:
flights.shape

In [0]:
## tidy2.pivot_table(index='name', columns='trt', values='result')
flights.pivot_table(index='year', columns='month', values='passengers')

# STRETCH OPTIONS

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
merged.head()